In [1]:
import os
import numpy as np
import cv2
from yolo_processor import Processor
from CLIP import ZeroShot
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import io
from PIL import Image

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
yolo = Processor(model_path="yolo11n.pt")
clip_checker = ZeroShot(saved_model_file_name = "fin_model(5).pt")
#proj_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

Use device: cpu
c:\Users\a\Desktop\CV-rare-animal\weights\fine-tuned\fin_model(5).pt
<class 'dict'>
Fine-tuned model loaded from c:\Users\a\Desktop\CV-rare-animal\weights\fine-tuned\fin_model(5).pt
Text features for shot ready.


In [3]:
upload = widgets.FileUpload(accept='.jpg,.jpeg,.png', multiple=False)
process_button = widgets.Button(description="Apply Detection", button_style='success')
reset_button = widgets.Button(description="Reset", button_style='danger')
output = widgets.Output()

img = None
def show_image():
    with output:
        clear_output(wait=False)
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        fig, ax = plt.subplots(1, 2, figsize=(12, 7))
        ax[0].imshow(rgb_img)
        ax[0].set_title("Original image")
        ax[0].axis('off')
        ax[1].axis('off')
        plt.tight_layout()
        plt.show()

def image_upload(change):
    global img
    if upload.value:
        content = list(upload.value.values())[0]['content']
        img = cv2.imdecode(np.frombuffer(content, np.uint8), cv2.IMREAD_COLOR)
        show_image()

def apply_process(b):
    global img
    if img is None:
        with output:
            clear_output()
            print("No image uploaded.")
        return

    detections, out_frame = yolo.run(img)
    is_rare = False
    message = "No animal found"
    res_image = img.copy()
    is_animal = False
    for det in detections:
        is_animal = True
        x1, y1, x2, y2 = det["bbox"]
        crop = det["crop"]
        results = clip_checker.classify_image(crop)
        top_res = results[0]
        label = top_res["label"]
        score = float(top_res["confidence"])

        if score > 0.3:
            color = (255, 255, 0)
            is_rare = True
            message = "Rare animal: " + label
        else:
            color = (0, 255, 0)
            label = "Normal"
            message = "No rare animal found"

        if (is_animal):
            if (is_rare):
                res_image = out_frame.copy()
        cv2.rectangle(res_image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(res_image, label, (x1 + 5, y1 + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
    
    with output:
        clear_output(wait=False)
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        fig, ax = plt.subplots(1, 2, figsize=(12, 7))
        ax[0].imshow(rgb_img)
        ax[0].set_title("Original image")
        ax[0].axis('off')
        ax[1].imshow(res_image)
        ax[1].set_title(message)
        ax[1].axis('off')
        plt.tight_layout()
        plt.show()

def reset_all(b):
    global img
    upload.value.clear()
    upload._counter = 0
    img = None
    with output:
        clear_output()




In [4]:
upload.observe(image_upload, names='value')
display(widgets.HTML("<h3>Simple proj</h3>"))
process_button.on_click(apply_process)
reset_button.on_click(reset_all)
low_col = widgets.HBox([process_button, reset_button])
# Display interface
display(widgets.VBox([upload, output, low_col]))

HTML(value='<h3>Simple proj</h3>')